# Projeto 1 - Ciência dos Dados

Nome: Eduardo Oliveira

Nome: Henrique Leite dos Santos

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\henri\OneDrive - Insper - Institudo de Ensino e Pesquisa\Aulas\Aulas 2º Semestre\CDados\Projeto 1\Projeto


Carregando a base de dados com os tweets classificados manualmente:

In [3]:
train_raw = pd.read_excel('dados_treino.xlsx')
train_raw.head(5)

,Mensagem,Target
0,O autor escreve sua experiência pessoal do que...,Livro
1,Partindo do fundamentalismo cristão e de ideia...,Livro
2,"Não li o livro ainda, então a avaliação refere...",Livro
3,"Não acompanho o youtube, talvez seja a falta d...",Livro
4,"O livro é apenas uma motivação, muito superfic...",Livro


In [4]:
test_raw = pd.read_excel('dados_teste.xlsx')
test_raw.head(5)

,Mensagem,Target
0,Acabei de comprar O Hobbit (versão Kindle) par...,amazon
1,"O diário de Anne Frank trata-se de uma farsa, ...",livro
2,Não achei muita graça não 👎🏻,livro
3,"Olá, Recebi uma notificação por email que não ...",amazon
4,"""Sapiens: uma breve história da humanidade rea...",livro


___
## Limpeza das mensagens


In [5]:
import re

# Função que limpa as pontuações das mensagens
def clean(text):
    punctuation = '[´"!-.:?;$'']'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

In [39]:
dados_train = []

# Limpeza das mensagens da base de dados de treino
for i in range(len(train_raw)):
    mensagem_raw = train_raw.Mensagem[i]
    mensagem = clean(mensagem_raw)
    target = train_raw.Target[i]
    
    dados_train.append([mensagem.lower(), target.lower()])
    
train = pd.DataFrame(dados_train, columns=['Mensagem','Target'])

train.head(5)

,Mensagem,Target
0,o autor escreve sua experiência pessoal do que...,livro
1,partindo do fundamentalismo cristão e de ideia...,livro
2,não li o livro ainda então a avaliação referes...,livro
3,não acompanho o youtube talvez seja a falta de...,livro
4,o livro é apenas uma motivação muito superfici...,livro


In [7]:
dados_test = []

# Limpeza das mensagens da base de dados de teste
for i in range(len(test_raw)):
    mensagem_raw = test_raw.Mensagem[i]
    mensagem = clean(mensagem_raw)
    target = test_raw.Target[i]
    
    dados_test.append([mensagem.lower(), target.lower()])
    
test = pd.DataFrame(dados_test, columns=['Mensagem','Target'])

test.head(5)

,Mensagem,Target
0,acabei de comprar o hobbit versão kindle para ...,amazon
1,o diário de anne frank tratase de uma farsa um...,livro
2,não achei muita graça não 👎🏻,livro
3,olá recebi uma notificação por email que não f...,amazon
4,sapiens uma breve história da humanidade realm...,livro


___
## Classificador automático


Faça aqui uma descrição do seu assunto e o que considerou como relevante ou não relevante na classificação das reviews (Target).

Ao definir a variável Target do projeto, nos colocamos na posição de um empregado da Amazon que deseja saber quais reclamações são em relação à entrega dos livros ou à plataforma Amazon (preços, entrega, etc.) e quais são reclamações relacionadas ao conteúdo dos livros.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

Eventos possíveis:

- $Amazon$: a reclamação é relacionada à Amazon;
- $Livro$: a reclamação é relacionada ao Livro;
- $Outro$: a reclamação não é relacionada à Amazon nem ao Livro;
- $Reclamação$: uma reclamação dentro da base de dados.


A partir das palavras da reclamação, queremos calcular a probabilidade da reclamação ser direcionada à Amazon, ao Livro ou outra coisa. Traduzindo em termos de probabilidades:

- $P(Amazon|Reclamação)$: probabilidade da reclamação ser para a "Amazon";
- $P(Livro|Reclamação)$: probabilidade da reclamação ser para o "Livro";
- $P(Outro|Reclamação)$: probabilidade da reclamação ser para "Outro".

A depender da maior probabilidade, dado a reclamação, classificaremos a mensagem como direcionada à alguma das partes.

As probabilidades serão calculadas da seguinte forma:

Para a $P(Amazon|Reclamação)$:

$$P(Amazon|Reclamação) = \frac{P(Reclamação|Amazon) P(Amazon)}{P(Reclamação)}$$

<br>

<br>

O processo é análogo para $P(Livro|Reclamação)$:

$$P(Livro|Reclamação) = \frac{P(Reclamação|Livro) P(Livro)}{P(Reclamação)}$$

<br>

<br>

E segue o mesmo para outros tipos de reclamação $P(Outro|Reclamação)$:

$$P(Outro|Reclamação) = \frac{P(Reclamação|Outro) P(Outro)}{P(Reclamação)}$$

<br>
<br>

Ou seja, precisamos de: 
- $P(Reclamação|Amazon)$: probabilidade de encontrar a reclamação na target Amazon;
- $P(Reclamação|Livro)$: probabilidade de encontrar a reclamação na target Livro;
- $P(Reclamação|Outro)$: probabilidade de encontrar a reclamação na target Livro;
- $P(Amazon)$: probabilidade da target ser "Amazon";
- $P(Livro)$: probabilidade da target ser "Livro";
- $P(Outro)$: probabilidade da target ser "Outro";
- $P(Reclamação)$: probabilidade de uma reclamação ocorrer na língua portuguesa.


### Probabilidades:
 - $P(livro)$
 - $P(amazon)$
 - $P(outro)$

In [12]:
P_target = train.Target.value_counts(normalize=True)

P_livro = P_target[0]
P_amazon = P_target[1]
P_outro = P_target[2]

In [43]:
mensagens_amazon = train.loc[train['Target']=='amazon', ['Mensagem']]
mensagens_amazon = mensagens_amazon.reset_index(drop=True)

todas_palavras_amazon = ''

for i in range(len(mensagens_amazon['Mensagem'])):
    msg = str(mensagens_amazon.Mensagem[i])
    todas_palavras_amazon += msg
    

serie_amazon = pd.Series(todas_palavras_amazon.split())

serie_amazon.head(10)

0            o
1      produto
2         veio
3        muito
4          bem
5     embalado
6            e
7            a
8    embalagem
9         para
dtype: object

In [50]:
tabela_amazon = serie_amazon.value_counts()
tabela_amazon_relativa = serie_amazon.value_counts(True)

tabela_amazon

o           90
a           82
de          78
que         76
e           69
            ..
algun        1
respeito     1
houvr        1
omitir       1
nest         1
Length: 930, dtype: int64

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CONSIDEROU mais de duas categorias na variável Target e INCREMENTOU a quantidade de notícias, mantendo pelo menos 250 notícias por categoria (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* Para Target com duas categorias: CRIOU pelo menos quatro categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item Qualidade do Classificador a partir de novas separações das Notícias entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**